In [2]:
import pandas as pd
import wikipedia as wp
from pygeocoder import Geocoder
import time
from googletrans import Translator
# Import and Initialize Sentiment Analyzer
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

import tweepy; import json
from apikeys import twitterAccessToken as access_token
from apikeys import twitterAccessTokenSecret as access_token_secret
from apikeys import twitterConsumerKey as consumer_key
from apikeys import twitterConsumerSecretKey as consumer_secret

def parse_url( url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    listylist=[]
    for table in soup.find_all('table'):
        listylist.append(parse_html_table(table))
    return listylist

def parse_html_table( table):
    n_columns = 0; n_rows=0; column_names = []

    # Find number of rows and columns
    # we also find the column titles if we can
    for row in table.find_all('tr'):

        # Determine the number of rows in the table
        td_tags = row.find_all('td')
        if len(td_tags) > 0:
            n_rows+=1
            if n_columns == 0:
                # Set the number of columns for our table
                n_columns = len(td_tags)

        # Handle column names if we find them
        th_tags = row.find_all('th') 
        if len(th_tags) > 0 and len(column_names) == 0:
            for th in th_tags:
                column_names.append(th.get_text())

    # Safeguard on Column Titles
    if len(column_names) > 0 and len(column_names) != n_columns:
        raise Exception("Column titles do not match the number of columns")

    columns = column_names if len(column_names) > 0 else range(0,n_columns)
    df = pd.DataFrame(columns = columns,
                      index= range(0,n_rows))
    row_marker = 0
    for row in table.find_all('tr'):
        column_marker = 0
        columns = row.find_all('td')
        for column in columns:
            df.iat[row_marker,column_marker] = column.get_text()
            column_marker += 1
        if len(columns) > 0:
            row_marker += 1

    # Convert to float if possible
    for col in df:
        try:
            df[col] = df[col].astype(float)
        except ValueError:
            pass

    return df

def getCountryLanguages():
    #TODO: Use the .apply to just change the table to one dialect. Imrpove language scope later.
    df = parse_url('https://www.infoplease.com/world/countries-world/languages-spoken-each-country-world')
    countryLanguages = df[0].rename(columns={0:'country',1:'language'}).set_index('country')
    countryLanguages['language'] = [re.sub('\d+|%|\(.*\)|\s','',i).split(',')[0].split(';')[0] for i in countryLanguages['language']]
    return countryLanguages


#returns hashtag, followers, following, text, geo, date
def SearchForData(search_term, nTweets, cityCountry='',radius=100):
    
    # Setup Tweepy API Authentication
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
    
    #--- Calculate geocordinates from cityCountry --- --- ---- ---- --- --- --- --
    try:
        result = Geocoder.geocode(cityCountry)
    except Exception as error:
        raise ValueError(f'Cannot translate to geo coordinates.\n',error)
    # 34.0934,56.134,50mi
    coords = str(result[0].coordinates).replace('(','').replace(')','') + f',{radius}mi'
    coords=coords.replace(' ','')
    print(cityCountry, ": ", coords)
    #--- ---- ----- ---- ---- ---- ---- ---- --- ---- ---- --- ---- --- ---- --- --

    #--- grab tweets --- ---- ---- ---- ---- ---- ---- ---- --- --- ---- ---- ----
    maxTweets = 10000; public_tweets=[]; oldest_tweet = None; unique_ids = []; desiredTweets = []
    while len(desiredTweets) < min(nTweets,maxTweets):
        #--- determine whether to grab tweets by geo or not --- ---- --- ----- --
        if cityCountry:
            public_tweets = api.search(search_term, count=100, result_type="recent", max_id=oldest_tweet, geocode=coords)
        else:
            public_tweets = api.search(search_term, count=100, result_type="recent", max_id=oldest_tweet)
        #---- ----- ----- ---- ----- ---- ----- ---- ----- ---- ---- ---- ---- --
        
        #--- Dont go through an infinite loop trying to fill tweets that don't exist -----
        if len(public_tweets['statuses'])==0:
            print(f'No tweets returned while searching for \'{search_term}\'')
            print(public_tweets['statuses'])
            return desiredTweets
        #--- --- ---- ---- ----- ---- ---- ---- ---- ---- ---- --- ---- ---- --- -- ------
        
        #--- Append relevent tweets to output listy list ---- --- ---- ---- ---- --- ---
        for tweet in public_tweets['statuses']:
            # Append tweet_id to ids list if it doesn't already exist. This allows checking for duplicate tweets
            if tweet["id"] not in unique_ids :
                unique_ids.append(tweet['id'])
                desiredTweets.append({'text':tweet['text'], 'geo':tweet['geo'], 
                                      'hashtags':tweet['entities']['hashtags'], 'followers':tweet['user']['followers_count'],
                                     'friends_count':tweet['user']['friends_count'],'statuses_count':tweet['user']['statuses_count']})
                                 
            # Reassign the the oldest tweet (i.e. the max_id) subtract 1 so the previous oldest isn't included
            oldest_tweet = tweet["id"] - 1

    #--- Print sample tweet --- --- ---- ---- --- ---- ---- --- ---- ---- ---
    translator = Translator()
    try:
        print ('Sample Tweet:',translator.translate(desiredTweets[0]['text'], dest='en').text)
    except:
        print('there was an error translating sample tweet: ',desiredTweets[0]['text'])
    return pd.DataFrame(desiredTweets)


def GetTweetsByPopularCities(search_term, numTweets):
    #-- Get the most populated cities from wikipedia (Thank you wikipedia library!) --
    html = wp.page("List_of_cities_by_population_density").html().encode("UTF-8")
    df = pd.read_html(html)[1]
    df=df.drop([2,3,4,5],axis=1)
    df=df.rename(columns={0:'city',1:'population',6:'country'})
    df=df.iloc[1:]
    #--- ---- ----- ----- ---- ---- ----- ---- ---- ----- --- ---- ---- ----- ---- ---

    translator = Translator()
    tweets={}
    for index,row in df.iterrows():
        #-- location --- ----- --- ----
        city,pop,country = row
        cityCountry = city+' , '+country

        #-- language conversion --- ---- --
        languagesDf = getCountryLanguages()
        try:
            translatedSearch = translator.translate(search_term, src='en', dest=languagesDf.loc[country,'language']).text
        except ValueError:
            print("could not translate ", languagesDf.loc[country,'language'])
            translatedSearch=search_term
        print('translated word: ',translatedSearch)
        tweets[city+','+country]=SearchForData(translatedSearch, numTweets, cityCountry, 100)
        print('\n')
        time.sleep(4)
        break
    return tweets

tweets = GetTweetsByPopularCities('dog', 100)
tweets{'Manila,Philippines'}


translated word:  aso
<class 'NoneType'>
Manila , Philippines :  14.5995124,120.9842195,100mi
there was an error translating sample tweet:  @ryapee Hi Rya order ako ulit polyblender mejo winasak ng aso ko. 😂




{'Manila,Philippines':     followers  following                                                geo  \
 0      154060      False                                               None   
 1          83      False                                               None   
 2         643      False                                               None   
 3        2729      False                                               None   
 4          25      False                                               None   
 5         116      False                                               None   
 6         113      False                                               None   
 7         130      False                                               None   
 8          54      False                                               None   
 9          24      False                                               None   
 10        841      False                                               None   
 11        260    

In [ ]:
from datetime import datetime
import matplotlib.pyplot as plt
timeBetween=[]
prevCreatedAt = datetime.strptime(tweets[0]['created_at'],'%a %b %d %H:%M:%S %z %Y')
for tweet in tweets[1:]:
    #print(tweet['created_at'])
    timeBetween.append((prevCreatedAt - datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y')).total_seconds()/60)
    prevCreatedAt = datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y')
plt.plot(range(999),timeBetween)
plt.show()
plt.plot([datetime.strptime(tweet['created_at'],'%a %b %d %H:%M:%S %z %Y') for tweet in tweets][::-1],range(1000))
#plt.x('date',rotation='vertical')
plt.show()
print((datetime.strptime(tweets[-1]['created_at'],'%a %b %d %H:%M:%S %z %Y')-datetime.strptime(tweets[0]['created_at'],'%a %b %d %H:%M:%S %z %Y')).total_seconds()//60//60)

In [ ]:
#This code came from the following link:
#https://srome.github.io/Parsing-HTML-Tables-in-Python-with-BeautifulSoup-and-pandas/
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

class HTMLTableParser():

    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        listylist=[]
        for table in soup.find_all('table'):
            listylist.append(self.parse_html_table(table))
        return listylist

    def parse_html_table(self, table):
        n_columns = 0; n_rows=0; column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):

            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)

            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1

        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass

        return df

#TODO: Use the .apply to just change the table to one dialect. Imrpove language scope later.
obj = HTMLTableParser()
df = obj.parse_url('https://www.infoplease.com/world/countries-world/languages-spoken-each-country-world')
countryLanguages = df[0].rename(columns={0:'country',1:'language'}).set_index('country')
newDict = []
for index, row in countryLanguages.iterrows():
    language = re.sub('\d+|%|\(.*\)|\s','',countryLanguages.loc[index].values[0]).split(',')[0].split(';')[0]
    newDict.append([index, language]) #print(index,": ",language)
newDict

newDf = pd.DataFrame(newDict)
newDf = newDf.rename(columns={0:'country',1:'language'}).set_index('country')
newDf

In [18]:
df = parse_url('https://www.infoplease.com/world/countries-world/languages-spoken-each-country-world')
countryLanguages = df[0].rename(columns={0:'country',1:'language'}).set_index('country')
countryLanguages['languages'] = [re.sub('\d+|%|\(.*\)|\s','',i).split(',')[0].split(';')[0] for i in countryLanguages['language']]
countryLanguages    

,language,languages
country,,
Afghanistan,"Dari Persian, Pashtu (both official), other Tu...",DariPersian
Albania,"Albanian (Tosk is the official dialect), Greek",Albanian
Algeria,"Arabic (official), French, Berber dialects",Arabic
Andorra,"Catalán (official), French, Castilian, Portuguese",Catalán
Angola,"Portuguese (official), Bantu and other African...",Portuguese
Antigua and Barbuda,"English (official), local dialects",English
Argentina,"Spanish (official), English, Italian, German, ...",Spanish
Armenia,"Armenian 98%, Yezidi, Russian",Armenian
Australia,"English 79%, native and other languages",English


In [ ]:
translator=Translator()
translator.translate('Hola me llamo Jennifer 😜😜', dest='en').text

In [34]:
# Setup Tweepy API Authentication
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())
tweets = api.search('dalailama')

In [35]:
tweets['statuses'][0]

{'contributors': None,
 'coordinates': None,
 'created_at': 'Fri Jun 29 01:52:56 +0000 2018',
 'entities': {'hashtags': [{'indices': [57, 63], 'text': 'peace'},
   {'indices': [87, 98], 'text': 'hatespeech'},
   {'indices': [100, 106], 'text': 'anger'}],
  'symbols': [],
  'urls': [{'display_url': 'twitter.com/i/web/status/1…',
    'expanded_url': 'https://twitter.com/i/web/status/1012514219374231553',
    'indices': [109, 132],
    'url': 'https://t.co/oSqPQyK9GL'}],
  'user_mentions': []},
 'favorite_count': 0,
 'favorited': False,
 'geo': None,
 'id': 1012514219374231553,
 'id_str': '1012514219374231553',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'place': None,
 'possibly_sensitive': False,
 'retweet_count': 0,
 'retweeted': False,
 'source': '<a href="

In [4]:
tweets['Manila,Philippines']

,followers,following,geo,hashtags,statuses_count,text
0,154060,False,None,[],22455,@ryapee Hi Rya order ako ulit polyblender mejo...
1,83,False,None,[],3030,Yung dating saling pusa naging aso bigla // 🎶
2,643,False,None,"[{'text': 'AdoptDontShop', 'indices': [38, 52]}]",8529,sana meron din dito sa Pilipinas yung #AdoptDo...
3,2729,False,None,[],28771,"Aso nga kasi ako, bantay ako dito hahaha 😂"
4,25,False,None,[],14,lakas mangahol ng kaklase ko dinaig pa aso nam...
5,116,False,None,[],1993,@LampanoElla Dun sa aso oo HAHA
6,113,False,None,[],4495,Me: labas mo dila para lumabas dila ng aso \nS...
7,130,False,None,[],1745,cute kong aso https://t.co/twDprm7o5P
8,54,False,None,[],727,me: pabili pong dog food\ntindero: alin? ung p...
9,24,False,None,[],296,@dsgalarpez hahahaha aso ka na ba ngayon?
